Read in synthetic lidar measurements from an AMR-Wind simulation, and reformat the data to look like real-world b-level measurements.

In [1]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import xarray as xr
from matplotlib.colors import TwoSlopeNorm
from pathlib import Path
import pandas as pd
import h5py

import dask.array as da

In [2]:
### Read in data and do some minimal processing
parent_dir = Path('/scratch/orybchuk/wakedynamics/bcs-ldm/simulations/072415/large_campaigns/precursor/post_processing')
# outdir = Path(parent_dir, 'reformatted')
outdir = Path('/scratch/orybchuk/wakedynamics/bcs-ldm/simulations/072415/large_campaigns/precursor/postprocessing', 'reformatted')
outdir.mkdir(exist_ok=True,parents=True)
t_offset = 43200

campaign_files = list(parent_dir.glob('sampling*.nc'))
campaign_files.sort()
n_campaigns = len(campaign_files)

# Common parameters

In [3]:
### Set up parameters common to all campaigns
## User inputs
varlist = ['velocityx', 'velocityy', 'velocityz']  # Manually build this
use_dask = False

## Data from simulations
with h5py.File(campaign_files[0]) as f_h5py:
    # Deal with time
    time = f_h5py['time'][:]
    time = np.round(time, 4)  # Round to deal with weird float behavior
    timestep = time[1] - time[0]
    
    # List of variables to process
    full_varlist = list(f_h5py['inflow-scan'].keys())
for var in varlist:
    assert var in full_varlist
print("varlist:", varlist)

varlist: ['velocityx', 'velocityy', 'velocityz']


# Reformat raw AMR-Wind lidar data to look like b-level measurements

In [4]:
### Reformat inflow boundary conditions
print("Start reformatting data", datetime.now())

for counter, fcampaign in enumerate(campaign_files[:100]):
# for counter, fcampaign in enumerate(campaign_files[320:]):
    if counter % 10 == 0: print(datetime.now(), '...', counter)
    
    campaign_id = fcampaign.name[8:12]
    
    ### ~~~~~~~~~~ Metadata for AMR-Wind inflow lidar ~~~~~~~~~~
    with Dataset(fcampaign) as f:
        in_time_table = f['inflow-scan'].time_table
        in_azimuth_table = f['inflow-scan'].azimuth_table      # NOTE: In practice, these may be shifted from the time_table timestamps
        in_elevation_table = f['inflow-scan'].elevation_table  # NOTE: In practice, these may be shifted from the time_table timestamps
        in_origin = f['inflow-scan'].start
        in_num_points = f['inflow-scan']['points'].shape[1]
        in_raw_coordinates = f['inflow-scan']['coordinates'][:].data  # These are just the coords at one moment in time
        in_raw_points = f['inflow-scan']['points'][:].data
        in_raw_velocityx = f['inflow-scan']['velocityx'][:].data
        in_raw_velocityy = f['inflow-scan']['velocityy'][:].data
        in_raw_velocityz = f['inflow-scan']['velocityz'][:].data

    in_length = np.sqrt((in_raw_coordinates[-1,0]-in_raw_coordinates[0,0])**2 \
                       +(in_raw_coordinates[-1,1]-in_raw_coordinates[0,1])**2 \
                       +(in_raw_coordinates[-1,2]-in_raw_coordinates[0,2])**2)
    in_length = np.round(in_length, 2)
    in_drange_gate = np.round(in_length/in_num_points, 2)
    in_range_gate = np.arange(0, in_length, in_drange_gate)
    time_datetime = np.array(pd.to_datetime(time, unit='s'))

    in_coords = {'time':time_datetime, 'range_gate': in_range_gate}
    in_attrs = {'Range gate length (m)': in_drange_gate,
                'Number of gates': float(in_num_points),
                'Scan type': 'RAAW inflow',
                'Filename': str(fcampaign),
                'Origin': in_origin}
    
    ### ~~~~~~~~~~ Create and populate Xarray Dataset of raw data ~~~~~~~~~~
    ds_in = xr.Dataset(coords=in_coords, attrs=in_attrs)

    ## Calculate positional data variables
    # Helper variables
    in_n_full_scans = int(len(time) / len(in_time_table))

    in_az_del_x = in_raw_points[:,-1,0] - in_raw_points[:,0,0]
    in_az_del_y = in_raw_points[:,-1,1] - in_raw_points[:,0,1]
    in_az_del_z = in_raw_points[:,-1,2] - in_origin[2]

    # Calculate angles
    in_azimuth = (np.rad2deg(np.arctan2(in_az_del_y, in_az_del_x))+360)%360
    in_azimuth = np.round(in_azimuth, 2)
    for unique_az in np.unique(in_azimuth):
        assert unique_az in in_azimuth_table, f"Azimuth {unique_az} not in the input azimuth table {in_azimuth_table}!"

    in_elevation = (np.rad2deg(np.arcsin(in_az_del_z/in_length))+360)%360
    in_elevation = np.round(in_elevation, 2)
    for unique_el in np.unique(in_elevation):
        assert unique_el in in_elevation_table, f"Elevation {unique_el} not in the input elevation table {in_elevation_table}!"

    in_pitch = np.zeros_like(in_azimuth)
    in_roll = np.zeros_like(in_azimuth)

    ## Calculate LOS velocity by projecting (u,v) onto the unit vector aligned with the azimuth
    ##  NOTE: By omitting elevation, I think this calc only works at 0 elevation
    in_unit_vec_az = np.vstack([np.cos(np.deg2rad(in_azimuth)), np.sin(np.deg2rad(in_azimuth))])
    assert np.all(np.isclose(np.sqrt(in_unit_vec_az[0]**2 + in_unit_vec_az[1]**2), 1)), "Unit vector mangitude doesn't equal 1!"
    in_wind_vec = np.stack((in_raw_velocityx, in_raw_velocityy))
    in_los_mag = np.sum(in_wind_vec * in_unit_vec_az[:,:,np.newaxis], axis=0)
    in_los_mag = np.abs(in_los_mag)
    assert np.abs(np.dot(in_wind_vec[:,-1,-1], in_unit_vec_az[:,-1])) == in_los_mag[-1,-1], "Error in vectorized dot product calculation!"

    ## Store in Dataset
    ds_in['azimuth'] = ('time', in_azimuth)
    ds_in['elevation'] = ('time', in_elevation)
    ds_in['pitch'] = ('time', in_pitch)
    ds_in['roll'] = ('time', in_roll)

    ds_in['los_wind_speed'] = (('time', 'range_gate'), in_los_mag)
    ds_in['velocityx_true'] = (('time', 'range_gate'), in_raw_velocityx)
    ds_in['velocityy_true'] = (('time', 'range_gate'), in_raw_velocityy)
    ds_in['velocityz_true'] = (('time', 'range_gate'), in_raw_velocityz)
    
    ### ~~~~~~~~~~ Match the real-world sampling frequency ~~~~~~~~~~
    ### Check some assumptions about the dataset format
    # assert ds_in['time'].values[1] - ds_in['time'].values[0] == np.timedelta64(500000000,'ns'), \
    #         "All the following postprocessing code assumes a 0.5 sec timestep!"
    assert np.timedelta64(1_000_000_000,'ns') % (ds_in['time'].values[1] - ds_in['time'].values[0]) == np.timedelta64(0,'ns'), \
            "All the following postprocessing code assumes that (1 sec) % (lidar timestep) == 0"
    n_lidar_meas_per_sec = int(np.timedelta64(1_000_000_000,'ns') / (ds_in['time'].values[1] - ds_in['time'].values[0]))
    for i in range(1+n_lidar_meas_per_sec):
        assert ds_in['azimuth'].values[i] == in_azimuth_table[0], \
            "All the following code assumes that the AMR-Wind beam position starts at the location of the azimuth table! Add more code to fix this!"
        
    ### Drop timestamps where we treat the scan as resetting
    skiptimes0 = ds_in['time'].where(ds_in['elevation'] == 45.0)
    keeptimes0_flag = (np.isnan(skiptimes0).values).astype(int)
    keeptimes0_inds = np.arange(0, len(keeptimes0_flag)) * keeptimes0_flag
    keeptimes0_inds = keeptimes0_inds[keeptimes0_flag != 0]

    ds_trim0 = ds_in.isel(time=keeptimes0_inds)

    ### Toss the first timestamp
    assert in_azimuth_table[0] != in_azimuth_table[n_lidar_meas_per_sec], "The following code is only valid when the 0th timestep is different than the 2nd timestep in the azimuth table"
    assert ds_trim0['azimuth'].values[0] == ds_trim0['azimuth'].values[n_lidar_meas_per_sec], "If this condition fails, then there's no need to run the below code"

    keeptimes1_inds = np.arange(1, len(ds_trim0['time']))
    ds_trim1 = ds_trim0.isel(time=keeptimes1_inds)
    
    ### Calculate coordinates for the downsampled dataset
    # if fcampaign == campaign_files[0]:
    #     print("WARNING: Manually downsampling to a frequency of 1 second, assuming an AMR-Wind sampling frequency of 0.5 seconds. The following code will not work for other timesteps.")
    assert len(np.unique(pd.DatetimeIndex(ds_trim1['time']).microsecond)) == n_lidar_meas_per_sec, "Unexpected number of microsecond values! Possible float rounding error?"
    skiptimes2 = ds_trim1['time'].where(pd.DatetimeIndex(ds_trim1['time']).microsecond  != 0)  # Keep timestamps with a microsecond value of 0, 
                                                                                               # effectively downsampling to 1 sec resolution
    keeptimes2_flag = (np.isnan(skiptimes2).values).astype(int)
    keeptimes2_inds = np.arange(0, len(keeptimes2_flag)) * keeptimes2_flag
    keeptimes2_inds = keeptimes2_inds[keeptimes2_flag != 0]

    keeptimes2 = ds_trim1['time'].isel(time=keeptimes2_inds).values
    coords2 = {'time': keeptimes2,
               'range_gate': ds_trim1['range_gate']}
    # keeptimes2

    ds_trim2 = xr.Dataset(coords=coords2)
    
    ### Collect time-averaged and instantaneous winds at our downsampled frequency
    az2 = np.zeros(len(ds_trim2['time']))
    el2 = np.zeros(len(ds_trim2['time']))
    wspd2_avg = np.zeros((len(ds_trim2['time']), 
                          len(ds_trim2['range_gate'])))
    velx2_avg = np.zeros((len(ds_trim2['time']), 
                          len(ds_trim2['range_gate'])))
    vely2_avg = np.zeros((len(ds_trim2['time']), 
                          len(ds_trim2['range_gate'])))
    velz2_avg = np.zeros((len(ds_trim2['time']), 
                          len(ds_trim2['range_gate'])))
    wspd2_inst = np.zeros((len(ds_trim2['time']), 
                          len(ds_trim2['range_gate'])))
    velx2_inst = np.zeros((len(ds_trim2['time']), 
                          len(ds_trim2['range_gate'])))
    vely2_inst = np.zeros((len(ds_trim2['time']), 
                          len(ds_trim2['range_gate'])))
    velz2_inst = np.zeros((len(ds_trim2['time']), 
                          len(ds_trim2['range_gate'])))

    for tstep in range(len(ds_trim1['time'])):
        curr_time = ds_trim1['time'].isel(time=tstep).values
        curr_az = ds_trim1['azimuth'].isel(time=tstep).values
        curr_el = ds_trim1['elevation'].isel(time=tstep).values
        curr_wspd = ds_trim1['los_wind_speed'].isel(time=tstep).values
        curr_velx = ds_trim1['velocityx_true'].isel(time=tstep).values
        curr_vely = ds_trim1['velocityy_true'].isel(time=tstep).values
        curr_velz = ds_trim1['velocityz_true'].isel(time=tstep).values

        # Identify if this is a 0 microsecond or 500 microsecond moment
        first_nonzero_microsecond_val = 1_000_000 / n_lidar_meas_per_sec
        if pd.DatetimeIndex([curr_time]).microsecond[0]  == first_nonzero_microsecond_val:  # This is a super ugly if statement, apologies to anyone reading this
            # avg_into_time = ds_trim1['time'].isel(time=tstep+1)
            # assert (ds_trim1['time'].isel(time=tstep+1) - ds_trim1['time'].isel(time=tstep)).values == np.timedelta64(500000000,'ns'), f"The next timestep {avg_into_time} should be 0.5 seconds away from the current time {curr_time}!"
            for sub_tstep in range(1,n_lidar_meas_per_sec):
                assert ds_trim1['azimuth'].isel(time=tstep+sub_tstep).values == ds_trim1['azimuth'].isel(time=tstep).values, f"The next azimuth {ds_trim1['azimuth'].isel(time=tstep+sub_tstep).values} should equal the current azimuth {ds_trim1['azimuth'].isel(time=tstep).values}!"
                assert ds_trim1['elevation'].isel(time=tstep+sub_tstep).values == ds_trim1['elevation'].isel(time=tstep).values, f"The next elevation {ds_trim1['elevation'].isel(time=tstep+sub_tstep).values} should equal the current elevation {ds_trim1['elevation'].isel(time=tstep).values}!"
                
            
            # Collect wind speeds for averaging
            # TODO: The below lines happen because we're hardcoding. Generalize this code.
            wspd_avg_list = [curr_wspd]
            velx_avg_list = [curr_velx]
            vely_avg_list = [curr_vely]
            velz_avg_list = [curr_velz]

        elif pd.DatetimeIndex([curr_time]).microsecond[0]  != 0:
            # Average wind speeds and save out winds
            wspd_avg_list.append(curr_wspd)
            velx_avg_list.append(curr_velx)
            vely_avg_list.append(curr_vely)
            velz_avg_list.append(curr_velz)
            
        else:
            # Average wind speeds and save out winds
            wspd_avg_list.append(curr_wspd)
            velx_avg_list.append(curr_velx)
            vely_avg_list.append(curr_vely)
            velz_avg_list.append(curr_velz)

    #         # Eyeball check that the two timesteps produce similar values
    #         print("WSPD")
    #         print(wspd_avg_list[0][-5:])
    #         print(wspd_avg_list[1][-5:])
    #         print()

    #         print("VELX")
    #         print(velx_avg_list[0][-5:])
    #         print(velx_avg_list[1][-5:])
    #         print()

            # # Do an extra check (this is a poor test for winds centered around 0)
            # for currlist in [wspd_avg_list, velx_avg_list, vely_avg_list, velz_avg_list]:
            #     assert (((currlist[0] - currlist[1]) / currlist[1]) * 100 < 10).all(), \
            #         "Difference between measured winds 0.5 sec apart exceeds 10%! That's unexpected"

            outstep = int(tstep / n_lidar_meas_per_sec)

            wspd2_avg[outstep, :] = np.mean(wspd_avg_list, axis=0)
            velx2_avg[outstep, :] = np.mean(velx_avg_list, axis=0)
            vely2_avg[outstep, :] = np.mean(vely_avg_list, axis=0)
            velz2_avg[outstep, :] = np.mean(velz_avg_list, axis=0)

            # Save out instantaneous winds
            wspd2_inst[outstep, :] = curr_wspd
            velx2_inst[outstep, :] = curr_velx
            vely2_inst[outstep, :] = curr_vely
            velz2_inst[outstep, :] = curr_velz        


    ## Store data in the Dataset
    ds_trim2['los_wspd_avg'] = (('time', 'range_gate'), wspd2_avg)
    ds_trim2['velx_true_avg'] = (('time', 'range_gate'), velx2_avg)
    ds_trim2['vely_true_avg'] = (('time', 'range_gate'), vely2_avg)
    ds_trim2['velz_true_avg'] = (('time', 'range_gate'), velz2_avg)
    ds_trim2['los_wspd_inst'] = (('time', 'range_gate'), wspd2_inst)
    ds_trim2['velx_true_inst'] = (('time', 'range_gate'), velx2_inst)
    ds_trim2['vely_true_inst'] = (('time', 'range_gate'), vely2_inst)
    ds_trim2['velz_true_inst'] = (('time', 'range_gate'), velz2_inst)
    
    ### Collect all non-velocity information for ds_trim2
    ds_trim2['azimuth'] = ds_trim1['azimuth'].sel(time=ds_trim2['time'].values)
    ds_trim2['elevation'] = ds_trim1['elevation'].sel(time=ds_trim2['time'].values)
    ds_trim2['pitch'] = ds_trim1['pitch'].sel(time=ds_trim2['time'].values)
    ds_trim2['roll'] = ds_trim1['roll'].sel(time=ds_trim2['time'].values)
    
    ### Trim off any dangling measurements
    n_unique_beams = len(np.unique(np.vstack((ds_trim2['azimuth'].values, ds_trim2['elevation'].values)).T, axis=0))
    n_whole_reps = np.floor(len(ds_trim2['time'])/n_unique_beams).astype(int)
    n_remainder_reps = len(ds_trim2['time']) % n_unique_beams
    if fcampaign == campaign_files[0]:
        print(f"Total number of whole reps: {n_whole_reps}; Number of extra beam measurements discarded: {n_remainder_reps}")

    ds_out = ds_trim2.isel(time=slice(0,n_unique_beams*n_whole_reps))
    ds_out.to_netcdf(Path(outdir,f'inflow_b1lev{campaign_id}.nc'))
    
print("End reformatting data", datetime.now())

Start reformatting data 2024-06-16 20:52:15.927492
2024-06-16 20:52:15.928657 ... 0
Total number of whole reps: 51; Number of extra beam measurements discarded: 0


KeyboardInterrupt: 

In [ ]:
counter

# Sanity check b-level data

In [ ]:
tmp0 = xr.open_dataset(Path(outdir,f'inflow_b1lev0000.nc'))
tmp1 = xr.open_dataset(Path(outdir,f'inflow_b1lev0001.nc'))
tmp2 = xr.open_dataset(Path(outdir,f'inflow_b1lev0002.nc'))

In [ ]:
tmp0

In [ ]:
tmp1